In [135]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
from pathlib import *
from pymongo import *

from exp.nb_00 import *
from exp.nb_01 import *
from exp.nb_02 import *
from exp.nb_03 import *

## Define your parameters here

In [141]:
param0 = LBLnewBestfitSWParam(band=9, vmin=8200, vmax=14290, nv=10000, dv=0.001,
                     molecule='h2o', conc='atmpro',
                     ref_pts=[(30, 250), (300, 250)], ng_refs=[3, 9], ng_adju=[-3, 0],
                     wgt=[(.95, .9, .6, .5), (.55, .55, .55, .65, .65, .7, .8, .9)],
                     cosz=.2588, rsfc=0,
                     klin=3e-25,
                     option_k_lookup=0,
                     atmpro='trp',
                     commitnumber='79f2292')

In [142]:
acs = [('saw', 1), ('trp', 0.2588)]

params = []
for atmpro, cosz in acs:
    dparam = vars(param0)
    dparam['atmpro'] = atmpro
    dparam['cosz'] = cosz
    params.append(LBLnewBestfitSWParam(**dparam))

In [143]:
params

[<class 'exp.nb_00.LBLnewBestfitSWParam'>
 {'dv': 0.001, 'nv': 10000, 'commitnumber': '79f2292', 'band': 9, 'molecule': 'h2o', 'atmpro': 'saw', 'tsfc': None, 'vmin': 8200, 'vmax': 14290, 'conc': 'atmpro', 'ref_pts': [(30, 250), (300, 250)], 'ng_refs': [3, 9], 'ng_adju': [-3, 0], 'wgt': [(0.95, 0.9, 0.6, 0.5), (0.55, 0.55, 0.55, 0.65, 0.65, 0.7, 0.8, 0.9)], 'cosz': 1, 'rsfc': 0, 'klin': 3e-25, 'option_k_lookup': 0},
 <class 'exp.nb_00.LBLnewBestfitSWParam'>
 {'dv': 0.001, 'nv': 10000, 'commitnumber': '79f2292', 'band': 9, 'molecule': 'h2o', 'atmpro': 'trp', 'tsfc': None, 'vmin': 8200, 'vmax': 14290, 'conc': 'atmpro', 'ref_pts': [(30, 250), (300, 250)], 'ng_refs': [3, 9], 'ng_adju': [-3, 0], 'wgt': [(0.95, 0.9, 0.6, 0.5), (0.55, 0.55, 0.55, 0.65, 0.65, 0.7, 0.8, 0.9)], 'cosz': 0.2588, 'rsfc': 0, 'klin': 3e-25, 'option_k_lookup': 0}]

## Run lblnew-bestfit-sw  (the fortran code)

In [144]:
WORKPLACE = Path('/chia_cluster/home/jackyu/radiation/analysis_-_new_kdist_param_sw/')

In [148]:
paths = [WORKPLACE/'/'.join(sorted(f'{n}_{v}' for n, v in vars(p).items())) for p in params]

PosixPath('/chia_cluster/home/jackyu/radiation/analysis_-_new_kdist_param_sw/atmpro_saw/band_9/commitnumber_79f2292/conc_atmpro/cosz_1/dv_0.001/klin_3e-25/molecule_h2o/ng_adju_[-3, 0]/ng_refs_[3, 9]/nv_10000/option_k_lookup_0/ref_pts_[(30, 250), (300, 250)]/rsfc_0/tsfc_None/vmax_14290/vmin_8200/wgt_[(0.95, 0.9, 0.6, 0.5), (0.55, 0.55, 0.55, 0.65, 0.65, 0.7, 0.8, 0.9)]')

In [149]:
runners = [LBLnewBestfitSWRun(path=path, param=param) for path, param in zip(paths, params)]

In [150]:
for runner in runners:
    runner.input_params()
    proc = runner.build()
    out, err = proc.communicate()
    runner.run()

Job <434165> is submitted to queue <serial>.

Job <434166> is submitted to queue <serial>.



In [151]:
runners[0].job_id

'434165'

In [152]:
runners[1].job_id

'434166'

In [153]:
! qstat -u jackyu

serial;  type=BATCH;  [ENABLED];  pri=35
4 run;   5 wait;

        REQUEST NAME        REQUEST ID          USER   STATE
   1:   fitsw_9_h2o_            434165       jackyu  RUNNING
   2:   fitsw_9_h2o_            434166       jackyu  RUNNING

v4q;  type=BATCH;  [ENABLED];  pri=30
96 run;   0 wait;


v4qexp;  type=BATCH;  [ENABLED];  pri=30
192 run;   0 wait;


LCCR_Q;  type=BATCH;  [ENABLED];  pri=30
16 run;   0 wait;


URBAN_Q;  type=BATCH;  [ENABLED];  pri=30
576 run;   0 wait;


orc;  type=BATCH;  [ENABLED];  pri=30
0 run;   0 wait;


RCEC_Q;  type=BATCH;  [ENABLED];  pri=30
144 run;   0 wait;


medium_priority;  type=BATCH;  [ENABLED];  pri=10
0 run;   0 wait;




## Analysis

In [126]:
analysis = LBLnewBestfitSWAnalysis(path=runner.path, runner=runner)

In [127]:
analysis.run()

In [128]:
proc = analysis.gitcommit()

Git-add  


In [129]:
out, err = proc.communicate()

In [130]:
print('Git-commit\n', f'stdout: {out.decode()}\n', f'stderr: {err.decode()}')

Git-commit
 stdout: [develop a1e1115] band09 h2o trp cosz=0.2588 nf_refs=[4, 8]
 1 file changed, 1841 insertions(+)
 create mode 100644 climatools/notebooks/sw_dev/atmpro_trp/band_9/commitnumber_79f2292/conc_atmpro/cosz_0.2588/dv_0.001/klin_3e-25/molecule_h2o/ng_adju_[-3, 0]/ng_refs_[4, 8]/nv_10000/option_k_lookup_0/ref_pts_[(30, 250), (300, 250)]/rsfc_0/tsfc_None/vmax_14290/vmin_8200/wgt_[(0.95, 0.9, 0.6, 0.55), (0.55, 0.55, 0.55, 0.6, 0.7, 0.75, 0.85, 0.95)]/analysis_-_lblnew-bestfit-sw.ipynb

 stderr: 


In [131]:
runner.path

PosixPath('atmpro_trp/band_9/commitnumber_79f2292/conc_atmpro/cosz_0.2588/dv_0.001/klin_3e-25/molecule_h2o/ng_adju_[-3, 0]/ng_refs_[4, 8]/nv_10000/option_k_lookup_0/ref_pts_[(30, 250), (300, 250)]/rsfc_0/tsfc_None/vmax_14290/vmin_8200/wgt_[(0.95, 0.9, 0.6, 0.55), (0.55, 0.55, 0.55, 0.6, 0.7, 0.75, 0.85, 0.95)]')

In [133]:
(PATH_ANALYSIS/runner.path).mkdir(parents=True, exists_ok=True)

PosixPath('/chia_cluster/home/jackyu/radiation/analysis_-_new_kdist_param_sw/atmpro_trp/band_9/commitnumber_79f2292/conc_atmpro/cosz_0.2588/dv_0.001/klin_3e-25/molecule_h2o/ng_adju_[-3, 0]/ng_refs_[4, 8]/nv_10000/option_k_lookup_0/ref_pts_[(30, 250), (300, 250)]/rsfc_0/tsfc_None/vmax_14290/vmin_8200/wgt_[(0.95, 0.9, 0.6, 0.55), (0.55, 0.55, 0.55, 0.6, 0.7, 0.75, 0.85, 0.95)]')

# fin